In [186]:
import PyPDF2 
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
import pandas as pd

porter = PorterStemmer()
pdfFileObj = open('/Users/akshayanagarajan/Downloads/Assignment/SpamDictionary.pdf', 'rb') 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
pageObj = pdfReader.getPage(0) 

spamDict = pageObj.extractText()
spamList = tokenizer.tokenize(spamDict)
spamList = [word.lower() for word in spamList]
spamList = spamList[2:]


#spamList = ['Free', 'Click here', 'visit', 'open attachment', 'call this number', 'money Out extra', 'Pension', 'Opportunity', 'Chance', 'Investment', 'Pension', 'offer', 'available']

wordDictD1 = dict.fromkeys(spamList, 0)
wordDictD2 = dict.fromkeys(spamList, 0)
wordDictD3 = dict.fromkeys(spamList, 0)
wordDictD4 = dict.fromkeys(spamList, 0)
wordDictD5 = dict.fromkeys(spamList, 0)
wordDictD6 = dict.fromkeys(spamList, 0)


stringToRemove = ['CMPE', '239', 'Ð', '02', 'Ð', 'SPAM', 'Document', 'Ð', 'for', 'generating', 'TF', '-IDF', 'Professor', 'Chandrasekar', 'Vuppalapati,', 'San', 'Jose', 'State', 'University', '<<<<<<', 'Note:', 'Actual', 'SPAM', 'MESSAGE', 'Ð', 'Do', 'not', 'click', 'or', 'follow', 'the', 'message', 'Ð', 'for', 'academic', 'analysis', 'purpose', 'ONLY.', '>>>>>>>>>']
D = []
tokenizer = RegexpTokenizer(r'\w+')

for i in range(1,7):
    pdfFileObj = open('/Users/akshayanagarajan/Downloads/Assignment/Document'+str(i)+'.pdf', 'rb') 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    pageObj = pdfReader.getPage(0) 
    Dict = pageObj.extractText()    
    tokenizer.tokenize(Dict)
    D.append(Dict.split())
flag = []   
for i in range(0,6):
    for word in stringToRemove:
        if word in D[i]:
            D[i].remove(word)       
    D[i] = [word for word in D[i] if word.isalpha()]
    D[i] = [word.lower() for word in D[i]]
    D[i] = [porter.stem(word) for word in D[i]]
    for word in D[i]:
        for spamWord in spamList:
            if spamWord == word :
                flag.append(True)

for word in D[0]:
    if word in wordDictD1:
        wordDictD1[word]+=1
for word in D[1]:
    if word in wordDictD2:
        wordDictD2[word]+=1
for word in D[2]:
    if word in wordDictD3:
        wordDictD3[word]+=1
for word in D[3]:
    if word in wordDictD4:
        wordDictD4[word]+=1
for word in D[4]:
    if word in wordDictD5:
        wordDictD5[word]+=1
for word in D[5]:
    if word in wordDictD6:
        wordDictD6[word]+=1        

wordDictDF = pd.DataFrame([wordDictD1, wordDictD2, wordDictD3, wordDictD4, wordDictD5, wordDictD6])

def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

tfBowA = computeTF(wordDictD1, D[0])
tfBowB = computeTF(wordDictD2, D[1])
tfBowC = computeTF(wordDictD3, D[2])
tfBowD = computeTF(wordDictD4, D[3])
tfBowE = computeTF(wordDictD5, D[4])
tfBowF = computeTF(wordDictD6, D[5])

def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        if val > 0:
            idfDict[word] = math.log10(N / float(val))
        
    return idfDict

idfs = computeIDF([wordDictD1, wordDictD2, wordDictD3, wordDictD4, wordDictD5, wordDictD6])

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

tfidfBowA = computeTFIDF(tfBowA, idfs)
tfidfBowB = computeTFIDF(tfBowB, idfs)
tfidfBowC = computeTFIDF(tfBowC, idfs)
tfidfBowD = computeTFIDF(tfBowD, idfs)
tfidfBowE = computeTFIDF(tfBowE, idfs)
tfidfBowF = computeTFIDF(tfBowF, idfs)

tfidfDF = pd.DataFrame([tfidfBowA, tfidfBowB, tfidfBowC, tfidfBowD, tfidfBowE, tfidfBowF])
print(tfidfDF)

pdfFileObj.close() 


       free     click      here     visit  open  attachment  call  this  \
0  0.005870  0.000000  0.000000  0.015904   0.0         0.0   0.0   0.0   
1  0.005870  0.000000  0.000000  0.015904   0.0         0.0   0.0   0.0   
2  0.006522  0.000000  0.000000  0.000000   0.0         0.0   0.0   0.0   
3  0.006072  0.013416  0.013416  0.000000   0.0         0.0   0.0   0.0   
4  0.000000  0.000000  0.000000  0.000000   0.0         0.0   0.0   0.0   
5  0.000000  0.000000  0.000000  0.000000   0.0         0.0   0.0   0.0   

   number     money      out     extra     offer  available   pension  \
0     0.0  0.000000  0.00000  0.000000  0.005870        0.0  0.000000   
1     0.0  0.000000  0.00000  0.000000  0.005870        0.0  0.000000   
2     0.0  0.000000  0.02882  0.000000  0.000000        0.0  0.057641   
3     0.0  0.000000  0.00000  0.000000  0.000000        0.0  0.000000   
4     0.0  0.008191  0.00000  0.016382  0.003707        0.0  0.000000   
5     0.0  0.000000  0.00000  0.0000